# Assignment 1

Deadline: 19.03.2026, 12:00 CET

<Add your name, student-id and emal address>

In [3]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance, is_pos_def, make_pos_def
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS
from helper_functions import simulate_correlated_gbm

c:\Users\cuve\Documents\Studium\Zürich\QuantPortfolioManagement\qpmwp-course\myenv\Lib\site-packages\qpsolvers\solvers\__init__.py:880: UserWarning: no QP solver found on your system, you can install solvers from PyPI by ``pip install qpsolvers[open_source_solvers]``
  warnings.warn(


## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a synthetic dataset of dimension TxN, T=1000, N=50, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [ ]:

# Set the dimensions
T = 1000       # Number of time steps
N = 50         # Number of assets
rnd_seed = 42  # Random seed for reproducibility

# Set random seed for reproducibility
np.random.seed(rnd_seed)

# Generate a random mean vector and covariance matrix.
# Make sure the covariance matrix is positive definite.

# mu = <your code here>
mu = np.random.uniform(0.05,0.2,N)
# sigma = <your code here>
A = np.random.randn(N,N)
sigma = A.dot(A.T)

# Generate correlated geometric Brownian motion paths and compute discrete returns
prices = simulate_correlated_gbm(mu=mu, sigma=sigma, T=T, random_seed=None)
returns = prices.pct_change().dropna()

# Compute the vector of expected returns from df using class ExpectedReturn
expected_returns = ExpectedReturn()
expected_returns.estimate(returns)
q = expected_returns.vector

# Compute the covariance matrix from df using class Covariance
covariance_matrix = Covariance()
covariance_matrix.estimate(returns)
P = covariance_matrix.matrix


# Display the results
print("Vector of expected returns (q):")
print(q)

print("\nCovariance matrix (P):")
print(P)

Vector of expected returns (q):
Asset_1    -0.029435
Asset_2    -0.025172
Asset_3    -0.022654
Asset_4    -0.029411
Asset_5    -0.023994
Asset_6    -0.012662
Asset_7    -0.013488
Asset_8    -0.022205
Asset_9    -0.023690
Asset_10   -0.019771
Asset_11   -0.013709
Asset_12   -0.023591
Asset_13   -0.033583
Asset_14   -0.019799
Asset_15   -0.030126
Asset_16   -0.022719
Asset_17   -0.031926
Asset_18   -0.023214
Asset_19   -0.022487
Asset_20   -0.025667
Asset_21   -0.024955
Asset_22   -0.023660
Asset_23   -0.045369
Asset_24   -0.035400
Asset_25   -0.015980
Asset_26   -0.007193
Asset_27   -0.007850
Asset_28   -0.033115
Asset_29   -0.031129
Asset_30   -0.033172
Asset_31   -0.012888
Asset_32   -0.029355
Asset_33   -0.017047
Asset_34   -0.033405
Asset_35   -0.026189
Asset_36   -0.007992
Asset_37   -0.019438
Asset_38   -0.020844
Asset_39   -0.030747
Asset_40   -0.028714
Asset_41   -0.015639
Asset_42   -0.025378
Asset_43   -0.031538
Asset_44   -0.025864
Asset_45   -0.026460
Asset_46   -0.036207
As

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the return series created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 15 assets is <= 0.3, the sum of assets 16 to 45 is <= 0.4 and the sum of assets 41 to 50 is <= 0.5

In [18]:
# Instantiate the Constraints class
constraints = Constraints(ids = returns.columns.tolist())

# Add budget constraint
# <your code here>
constraints.add_budget(rhs=1, sense='=')

# Add box constraints (i.e., lower and upper bounds)
#<your code here>
constraints.add_box(box_type='LongOnly', lower=0, upper=0.2)

# Add linear constraints
# <your code here>

g1 = pd.Series(0.0, index=returns.columns)
g1.iloc[0:15] = 1.0
constraints.add_linear(g_values=g1, sense='<=', rhs=0.3, name='group1')

g2 = pd.Series(0.0, index=returns.columns)
g2.iloc[15:40] = 1.0
constraints.add_linear(g_values=g2, sense='<=', rhs=0.4, name='group2')

g3 = pd.Series(0.0, index=returns.columns)
g3.iloc[40:50] = 1.0
constraints.add_linear(g_values=g3, sense='<=', rhs=0.5, name='group3')


# Display some columns of the G matrix to verify that the constraints have been set up correctly
constraints.linear['G'][['Asset_1', 'Asset_15', 'Asset_16', 'Asset_40', 'Asset_41', 'Asset_50']]

,Asset_1,Asset_15,Asset_16,Asset_40,Asset_41,Asset_50
group1,1.0,1.0,0.0,0.0,0.0,0.0
group2,0.0,0.0,1.0,1.0,0.0,0.0
group3,0.0,0.0,0.0,0.0,1.0,1.0


### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers that you could install and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constraints fulfilled? Extract primal residuals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers where the optimization failed for some reason.




In [ ]:
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

import time

# Prepare problem data
P_array = (np.array(P) + np.array(P).T) / 2  # Ensure symmetry
q_array = np.array(q)
lb = np.array(constraints.box['lower'])
ub = np.array(constraints.box['upper'])
A = GhAb['A'].reshape(1, -1) if GhAb['A'] is not None else None
b = np.array(GhAb['b']).flatten() if GhAb['b'] is not None else None

# Define a dictionary to store the results in case a solver fails
result_on_fail = {
    'solution_found': False,
    'objective': np.nan,
    'primal_residual': np.nan,
    'dual_residual': np.nan,
    'duality_gap': np.nan,
    'runtime': np.nan,
}

results = {}

for solver in USABLE_SOLVERS:
    try:
        qp = QuadraticProgram(
            P=P_array,
            q=q_array,
            G=GhAb['G'],
            h=GhAb['h'],
            A=A,
            b=b,
            lb=lb,
            ub=ub,
            solver=solver
        )

        start = time.time()
        qp.solve()
        runtime = time.time() - start

        sol = qp.results['solution']

        if not sol.found:
            results[solver] = result_on_fail.copy()
        else:
            results[solver] = {
                'solution_found': sol.found,
                'objective': qp.objective_value(),
                'primal_residual': sol.primal_residual(),
                'dual_residual': sol.dual_residual(),
                'duality_gap': sol.duality_gap(),
                'runtime': runtime,
            }

    except Exception as e:
        print(f"Solver '{solver}' failed: {e}")
        results[solver] = result_on_fail.copy()

# Build results DataFrame
df_results = pd.DataFrame(results, index=['solution_found', 'objective', 'primal_residual', 'dual_residual', 'duality_gap', 'runtime'])

print(df_results)

# Loop over solvers, instantiate the quadratic program, solve it and store the results in a dictionary.


Solver 'quadprog' failed: 'quadprog' does not seem to be installed (found solvers: []); if 'quadprog' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can install it by `pip install qpsolvers[quadprog]`
Solver 'osqp' failed: 'osqp' does not seem to be installed (found solvers: []); if 'osqp' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can install it by `pip install qpsolvers[osqp]`
Solver 'daqp' failed: 'daqp' does not seem to be installed (found solvers: []); if 'daqp' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can install it by `pip install qpsolvers[daqp]`
Solver 'qpalm' failed: 'qpalm' does not seem to be installed (found solvers: []); if 'qpalm' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can install it by `pip install qpsolvers[qpalm]`
Solver 'cvxopt' failed: 'cvxopt' does not seem to be installed (found solvers: []); if 'cvxopt' is listed in https://github.com/qpsolvers/qpsolvers#solvers you can install 

Print and visualize the results

In [5]:
# <your code here>

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        # <your code here>

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            # <your code here>
            return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
# <your code here>

# Instantiate the MinVariance class
# <your code here>

# Prepare the optimization data and prepare the optimization problem
# <your code here>

# Solve the optimization problem and print the weights
# <your code here>